<a href="https://colab.research.google.com/github/langtheworld/Mukapon-Py1/blob/main/XGBoosting_MidTerm_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gensim

In [2]:
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#1. Load data

In [3]:
# Replace 'your_file.json' with the actual file name
trainfile_name = 'train_for_student.json'

# Open and read the JSON file
with open(trainfile_name, 'r') as json_file:
    traindata = json.load(json_file)

In [4]:
pd.DataFrame(traindata)

,001,002,003,004,005,006,007,008,009,010,...,445,446,447,448,449,450,451,452,453,454
Title,Activated carbon derived from bacterial cellul...,The algorithm of static hand gesture recogniti...,Alternative Redundant Residue Number System Co...,Comparative study of wax inhibitor performance...,Undrained lower bound solutions for end bearin...,Words Diffusion an Analysis of across Facebook...,Transformation of time Petri net into Promela,Annual Degradation Rate Analysis of Mono-Si Ph...,Development of Low-Cost in-the-Ear EEG Prototype,Model-based analysis of an integrated zinc-air...,...,Effect of rhenium and cobalt additions on the ...,Wheeling charge calculation with consideration...,Scaling laws for static displacement of linear...,Effect of TMB/P123 ratios on physicochemical p...,Synthetic CaO-based sorbent for high-temperatu...,A portable USB-controlled potentiostat for pap...,Literature reviews on applying artificial inte...,A multi-parameterized water quality prediction...,Semantic Segmentation on Medium-Resolution Sat...,Reducing the defects of a-pillar stamping part...
Abstract,© 2019 Elsevier B.V.Activated carbon derived f...,© Springer International Publishing AG 2018.Te...,© 2018 IEEE.Residue number system (RNS) is a n...,© Published under licence by IOP Publishing Lt...,"© 2019 John Wiley & Sons, Ltd.The undrained be...",© 2018 IEEE.Facebook Pages in Thailand have be...,© 2017 IEEE.This paper proposes a method of tr...,© 2013 IEEE.The annual degradation rate (DR) o...,© 2018 IEEE.This study focused on building a l...,"© 2019 Lao-atiman, Bumroongsil, Arpornwichanop...",...,© Carl Hanser Verlag GmbH & Co. KG.The effect ...,"© 2019 IEEE.In Thailand, Small Power Producers...",© 2018 Elsevier LtdExperimental studies on the...,© 2021 Elsevier LtdThe preparation of mesocell...,© 2018 Hydrogen Energy Publications LLCCalcium...,© 2018 IEEEThis paper presents a portable and ...,Copyright © 2019 for this paper by its authors...,© 2019 The authors and IOS Press. All rights r...,© 2018 IEEE.Semantic Segmentation is a fundame...,© 2019 IEEE.This research aims to reduce defec...
Classes,"[CHE, MATENG]",[CPE],[EE],"[PE, ME, CHE]","[CE, MATSCI]",[CPE],"[CPE, MATH]","[PE, EE, CHE]","[BME, CPE, IE]","[PE, METAL, EE, CPE, CHE, IE, MATH]",...,"[METAL, ME, CHE, MATH, MATSCI]","[ME, EE]","[CE, MATSCI]","[OPTIC, CHE, MATSCI]","[PE, CHE]","[CPE, CHE]","[CPE, EDU]","[ENV, EE, CHE]","[EE, CPE, OPTIC, EDU]","[METAL, EDU, MATSCI]"


In [5]:
# format to data table
data = pd.DataFrame(traindata).T.reset_index()

data

,index,Title,Abstract,Classes
0,001,Activated carbon derived from bacterial cellul...,© 2019 Elsevier B.V.Activated carbon derived f...,"[CHE, MATENG]"
1,002,The algorithm of static hand gesture recogniti...,© Springer International Publishing AG 2018.Te...,[CPE]
2,003,Alternative Redundant Residue Number System Co...,© 2018 IEEE.Residue number system (RNS) is a n...,[EE]
3,004,Comparative study of wax inhibitor performance...,© Published under licence by IOP Publishing Lt...,"[PE, ME, CHE]"
4,005,Undrained lower bound solutions for end bearin...,"© 2019 John Wiley & Sons, Ltd.The undrained be...","[CE, MATSCI]"
...,...,...,...,...
449,450,A portable USB-controlled potentiostat for pap...,© 2018 IEEEThis paper presents a portable and ...,"[CPE, CHE]"
450,451,Literature reviews on applying artificial inte...,Copyright © 2019 for this paper by its authors...,"[CPE, EDU]"
451,452,A multi-parameterized water quality prediction...,© 2019 The authors and IOS Press. All rights r...,"[ENV, EE, CHE]"
452,453,Semantic Segmentation on Medium-Resolution Sat...,© 2018 IEEE.Semantic Segmentation is a fundame...,"[EE, CPE, OPTIC, EDU]"


#2. Data preprocessing

##2.1 Cleansing data text แสดงผลเฉพาะตัวอักษรออกมา

In [6]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Remove Number
    s = re.sub('[0-9]+', '', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\!\?\(\)\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    return s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
# add new column which is Title+Abstract
data['TitleAbstract'] =  data['Title'] + data['Abstract']

In [8]:
data['Title_clean'] = data['Title'].apply(text_preprocessing)
data['Abstract_clean'] = data['Abstract'].apply(text_preprocessing)

In [9]:
# Cleansing data in TitleAbstract
data['TitleAbstract_clean'] = data['TitleAbstract'].apply(text_preprocessing)

##2.2 Transform Y data

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

In [11]:
# Create a MultiLabelBinarizer instance
mlb = MultiLabelBinarizer()

In [12]:
# Fit and transform the labels into a binary matrix
one_hot_labels = mlb.fit_transform(data['Classes'])

# Create a new DataFrame with one-hot encoded labels
one_hot_df = pd.DataFrame(one_hot_labels, columns=mlb.classes_)

# Connect Column one hot label together
data_clean = pd.concat([data, one_hot_df], axis=1)

In [13]:
data_clean

,index,Title,Abstract,Classes,TitleAbstract,Title_clean,Abstract_clean,TitleAbstract_clean,AGRI,BME,...,IE,MATENG,MATH,MATSCI,ME,METAL,NANO,OPTIC,PE,SAFETY
0,001,Activated carbon derived from bacterial cellul...,© 2019 Elsevier B.V.Activated carbon derived f...,"[CHE, MATENG]",Activated carbon derived from bacterial cellul...,activated carbon derived bacterial cellulose u...,elsevier b v activated carbon derived bacteria...,activated carbon derived bacterial cellulose u...,0,0,...,0,1,0,0,0,0,0,0,0,0
1,002,The algorithm of static hand gesture recogniti...,© Springer International Publishing AG 2018.Te...,[CPE],The algorithm of static hand gesture recogniti...,algorithm static hand gesture recognition usin...,springer international publishing ag technolog...,algorithm static hand gesture recognition usin...,0,0,...,0,0,0,0,0,0,0,0,0,0
2,003,Alternative Redundant Residue Number System Co...,© 2018 IEEE.Residue number system (RNS) is a n...,[EE],Alternative Redundant Residue Number System Co...,alternative redundant residue number system co...,ieee residue number system rns number represen...,alternative redundant residue number system co...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,004,Comparative study of wax inhibitor performance...,© Published under licence by IOP Publishing Lt...,"[PE, ME, CHE]",Comparative study of wax inhibitor performance...,comparative study wax inhibitor performance po...,published licence iop publishing ltd petroleum...,comparative study wax inhibitor performance po...,0,0,...,0,0,0,0,1,0,0,0,1,0
4,005,Undrained lower bound solutions for end bearin...,"© 2019 John Wiley & Sons, Ltd.The undrained be...","[CE, MATSCI]",Undrained lower bound solutions for end bearin...,undrained lower bound solutions end bearing ca...,john wiley sons ltd undrained bearing capacity...,undrained lower bound solutions end bearing ca...,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,450,A portable USB-controlled potentiostat for pap...,© 2018 IEEEThis paper presents a portable and ...,"[CPE, CHE]",A portable USB-controlled potentiostat for pap...,portable usb controlled potentiostat paper bas...,ieeethis paper presents portable inexpensive u...,portable usb controlled potentiostat paper bas...,0,0,...,0,0,0,0,0,0,0,0,0,0
450,451,Literature reviews on applying artificial inte...,Copyright © 2019 for this paper by its authors...,"[CPE, EDU]",Literature reviews on applying artificial inte...,literature reviews applying artificial intelli...,copyright paper authors use permitted creative...,literature reviews applying artificial intelli...,0,0,...,0,0,0,0,0,0,0,0,0,0
451,452,A multi-parameterized water quality prediction...,© 2019 The authors and IOS Press. All rights r...,"[ENV, EE, CHE]",A multi-parameterized water quality prediction...,multi parameterized water quality prediction m...,authors ios press rights reserved paper presen...,multi parameterized water quality prediction m...,0,0,...,0,0,0,0,0,0,0,0,0,0
452,453,Semantic Segmentation on Medium-Resolution Sat...,© 2018 IEEE.Semantic Segmentation is a fundame...,"[EE, CPE, OPTIC, EDU]",Semantic Segmentation on Medium-Resolution Sat...,semantic segmentation medium resolution satell...,ieee semantic segmentation fundamental task co...,semantic segmentation medium resolution satell...,0,0,...,0,0,0,0,0,0,0,1,0,0


#2.3 TF-IDF:Word to Vector

In [14]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
data_preprocessed = np.array(data_clean['Title_clean'])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)
data_train_tfidf = tf_idf.fit_transform(data_preprocessed)

CPU times: user 27.4 ms, sys: 927 µs, total: 28.4 ms
Wall time: 38.7 ms


#3. Model

In [15]:
import xgboost as xgb

In [16]:
# Create an XGBoost multi-label classifier
clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False  # XGBoost 1.0+ doesn't require label encoding
)

In [17]:
# Fit the model to the training data
clf.fit(data_train_tfidf, data_clean.loc[:,'AGRI':])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#Test data

In [18]:
# Replace 'your_file.json' with the actual file name
testFIle_name = 'test_for_student.json'

# Open and read the JSON file
with open(testFIle_name, 'r') as json_file:
    testdata = json.load(json_file)

In [19]:
# format to data table
test_data = pd.DataFrame(testdata).T

test_data

,Title,Abstract
001eval,Comparative Electrical Energy Yield Performanc...,© 2013 IEEE.Long-term energy evaluation of PV ...
002eval,Effects of graphene nanoplatelets on bio-based...,© The Author(s) 2021.Novel near-infrared (NIR)...
003eval,Anti-inflammatory action of two novel peptides...,© The Royal Society of Chemistry 2020.Peanut w...
004eval,Efficient all-and-one support vector machines ...,© 2018 IEEE.We introduce a new strategy to est...
005eval,Driver identification using histogram and neur...,© 2017 IEEE.Sensor technology has continuously...
...,...,...
147eval,Utilization of Sewage Sludge from Beverage Ind...,© Published under licence by IOP Publishing Lt...
148eval,Development of a Gateway for OpenADR-ECHONET L...,"© 2018 IEEE.In this paper, we develop an ECHON..."
149eval,Effect of solution treatment and precipitation...,© 2017 Elsevier Ltd. All rights reserved.The a...
150eval,An effect-analysis method for species-dependen...,"© The Authors, published by EDP Sciences, 2019..."


In [20]:
# Preprocessing Test data
test_data['TitleAbstract_clean'] = test_data['Title'] + test_data['Abstract']
test_data['Title_clean'] = test_data['Title'].apply(text_preprocessing)
test_data['Title_clean'] = np.array(test_data['Title'].apply(text_preprocessing))

#Array abstract
test_data['Abstract_clean'] = test_data['Abstract'].apply(text_preprocessing)
test_data['Abstract_clean'] = np.array(test_data['Abstract'].apply(text_preprocessing))
test_data['TitleAbstract_clean'] = np.array(test_data['TitleAbstract_clean'].apply(text_preprocessing))

data_test = tf_idf.transform(test_data['Abstract_clean'])

In [21]:
# Make predictions on the test data
y_pred = clf.predict(data_test)

In [22]:
data_pred = pd.DataFrame(y_pred, index=test_data.index, columns=['AGRI',
       'BME', 'CE', 'CHE', 'CPE', 'EDU', 'EE', 'ENV', 'IE', 'MATENG', 'MATH',
       'MATSCI', 'ME', 'METAL', 'NANO', 'OPTIC', 'PE', 'SAFETY'])

data_pred

,AGRI,BME,CE,CHE,CPE,EDU,EE,ENV,IE,MATENG,MATH,MATSCI,ME,METAL,NANO,OPTIC,PE,SAFETY
001eval,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002eval,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
003eval,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
004eval,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
005eval,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147eval,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
148eval,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149eval,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
150eval,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
desired_columns = ['CE', 'ENV', 'BME', 'PE', 'METAL', 'ME', 'EE', 'CPE', 'OPTIC', 'NANO', 'CHE', 'MATENG', 'AGRI', 'EDU', 'IE', 'SAFETY', 'MATH', 'MATSCI']
data_pred = data_pred[desired_columns]
data_pred = data_pred.astype(int)
data_pred.insert(0, 'id', test_data.index)
data_pred

,id,CE,ENV,BME,PE,METAL,ME,EE,CPE,OPTIC,NANO,CHE,MATENG,AGRI,EDU,IE,SAFETY,MATH,MATSCI
001eval,001eval,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
002eval,002eval,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
003eval,003eval,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
004eval,004eval,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
005eval,005eval,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147eval,147eval,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,1
148eval,148eval,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
149eval,149eval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
150eval,150eval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#save To csv เพื่อส่งให้อาจารย์ตรวจ

In [30]:
data_pred.to_csv('test_exam_midterm.csv', index=False)

In [31]:
data_clean.to_csv('data_clean_input.csv')

In [32]:
# Save Model
clf.save_model('xgboost_model.model')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:07:07] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
